In [29]:
from math import exp
from random import seed
from random import random
import numpy as np

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
  network = list()
  hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
  network.append(hidden_layer)
  output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
  network.append(output_layer)
  return network

# Calculate neuron activation for an input
def activate(weights, inputs):
  activation = weights[-1]
  for i in range(len(weights)-1):
    activation += weights[i] * inputs[i]
  return activation

# Transfer neuron activation
def transfer(activation):
  return 1.0 / (1.0 + exp(-activation))

# Forward propagate input to a network output
def forward_propagate(network, row):
  inputs = row
  for layer in network:
    new_inputs = []
    for neuron in layer:
      activation = activate(neuron['weights'], inputs)
      neuron['output'] = transfer(activation)
      new_inputs.append(neuron['output'])
    inputs = new_inputs
  return inputs

# Calculate the derivative of an neuron output
def transfer_derivative(output):
  return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
  for i in reversed(range(len(network))):
    layer = network[i]
    errors = list()
    if i != len(network)-1:
      for j in range(len(layer)):
        error = 0.0
        for neuron in network[i + 1]:
          error += (neuron['weights'][j] * neuron['delta'])
        errors.append(error)
    else:
      for j in range(len(layer)):
        neuron = layer[j]
        errors.append(expected[j] - neuron['output'])
    for j in range(len(layer)):
      neuron = layer[j]
      neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

# Update network weights with error
def update_weights(network, row, l_rate):
  for i in range(len(network)):
    inputs = row[:-1]
    if i != 0:
      inputs = [neuron['output'] for neuron in network[i - 1]]
    for neuron in network[i]:
      for j in range(len(inputs)):
        neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
      neuron['weights'][-1] += l_rate * neuron['delta']

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
  for epoch in range(n_epoch):
    sum_error = 0
    for row in train:
      outputs = forward_propagate(network, row)
      expected = [0 for i in range(n_outputs)]
      expected[row[-1]] = 1
      sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
      backward_propagate_error(network, expected)
      update_weights(network, row, l_rate)
    print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [7]:
print("Loading Train_x . . .")
train_x = np.loadtxt("train_x_processed2c.csv", delimiter=",") # load from text 

print("Loading Train_y . . .")
train_y = np.loadtxt("train_y.csv", delimiter=",")

print("Loading Test_X . . .")
test_x = np.loadtxt("test_x_processed2c.csv", delimiter=",")

Loading Train_x . . .
Loading Train_y . . .
Loading Test_X . . .


In [71]:
from tqdm import tqdm

input_trainX = []
tempX = train_x//255
for i in tqdm(range(len(tempX))):
    row = []
    for j in range(len(tempX[0])):
        row.append(tempX[i][j])
    row.append(int(train_y[i]))
    input_trainX.append(row)

100%|██████████| 50000/50000 [01:19<00:00, 627.60it/s]


In [72]:
# Test training backprop algorithm

seed(1)
dataset = input_trainX

n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
    print(layer)

>epoch=0, lrate=0.500, error=42419.841
>epoch=1, lrate=0.500, error=41718.420
>epoch=2, lrate=0.500, error=41521.823
>epoch=3, lrate=0.500, error=41416.217
>epoch=4, lrate=0.500, error=41330.300
>epoch=5, lrate=0.500, error=40864.579
>epoch=6, lrate=0.500, error=40746.464
>epoch=7, lrate=0.500, error=40169.971
>epoch=8, lrate=0.500, error=39656.973
>epoch=9, lrate=0.500, error=39545.286
>epoch=10, lrate=0.500, error=39537.470
>epoch=11, lrate=0.500, error=39459.618
>epoch=12, lrate=0.500, error=39392.682
>epoch=13, lrate=0.500, error=39365.704
>epoch=14, lrate=0.500, error=39339.165
>epoch=15, lrate=0.500, error=39319.055
>epoch=16, lrate=0.500, error=39296.313
>epoch=17, lrate=0.500, error=39265.758
>epoch=18, lrate=0.500, error=39283.674
>epoch=19, lrate=0.500, error=39211.214
[{'weights': [0.13436424411240122, 0.84743373693723267, 0.76377461897661403, 0.2550690257394217, 0.49543508709194095, 0.44949106478873813, 0.65159297272276295, 0.78872335113551317, 0.093859586774234893, 0.02834